This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#Note- I'm commenting this out for checking into git and pushing to github. Printing my API key here is just as bad as pushing my .env file!
#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json
import csv

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
start_date="2017-05-24"
end_date="2017-05-24"
url=url=f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date={start_date}&end_date={end_date}&api_key={API_KEY}"
try:
    test_response = requests.get(url, timeout=5)
    test_response.raise_for_status()  # Raises an HTTPError for bad responses (4XX or 5XX)
    test_data = test_response.json()
except requests.HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')  # Specific HTTP error handling
except requests.Timeout as timeout_err:
    print(f'Timeout error occurred: {timeout_err}')  # Specific timeout handling
except requests.ConnectionError as conn_err:
    print(f'Connection error occurred: {conn_err}')  # Specific connection error handling
except requests.RequestException as req_err:
    print(f'Error during request to {url}: {req_err}')  # Catch-all for any Request issues
except Exception as e:
    print(f'An error occurred: {e}')  # Optional: catch-all for any other errors
print("Data successfully retrieved")

Data successfully retrieved


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json.dumps(test_data, indent=4))

{
    "dataset_data": {
        "limit": null,
        "transform": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "start_date": "2017-05-24",
        "end_date": "2017-05-24",
        "frequency": "daily",
        "data": [
            [
                "2017-05-24",
                43.92,
                44.67,
                43.92,
                44.53,
                null,
                111923.0,
                4971343.0,
                null,
                null,
                null
            ]
        ],
        "collapse": null,
        "order": null
    }
}


Notes about above data structure.
Everything is nested in 'dataset_data'
The 'columns_names' key is useful as this tells me that the values are in the list.
The 'data' key has a list, for each day of the data (in the order of the 'column names')
These are the two things I'll need. 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
# Convert the returned JSON object into a Python dictionary.
start_date="2017-01-01"
end_date="2017-12-31"
url=url=f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date={start_date}&end_date={end_date}&api_key={API_KEY}"
try:
    response = requests.get(url, timeout=5)
    response.raise_for_status()  # Raises an HTTPError for bad responses (4XX or 5XX)
    AFX_X_data = response.json()
except requests.HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')  # Specific HTTP error handling
except requests.Timeout as timeout_err:
    print(f'Timeout error occurred: {timeout_err}')  # Specific timeout handling
except requests.ConnectionError as conn_err:
    print(f'Connection error occurred: {conn_err}')  # Specific connection error handling
except requests.RequestException as req_err:
    print(f'Error during request to {url}: {req_err}')  # Catch-all for any Request issues
except Exception as e:
    print(f'An error occurred: {e}')  # Optional: catch-all for any other errors
print("Data successfully retrieved")

Data successfully retrieved


In [6]:
#Confirm dates
print(f"Date Range: {AFX_X_data['dataset_data']['start_date']} - {AFX_X_data['dataset_data']['end_date']}")

Date Range: 2017-01-01 - 2017-12-31


In [7]:
#explore data a bit
data=AFX_X_data['dataset_data']['data']
#sort sublists so they are in chronological order
data.reverse()
print(f"Number of days: {len(data)}")

Number of days: 255


In [8]:
print(data[:10])

[['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None], ['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-04', 35.48, 35.51, 34.75, 35.19, None, 54408.0, 1906810.0, None, None, None], ['2017-01-05', 35.02, 35.2, 34.73, 35.06, None, 48412.0, 1692326.0, None, None, None], ['2017-01-06', 34.91, 35.21, 34.91, 35.04, None, 27507.0, 964046.0, None, None, None], ['2017-01-09', 35.29, 35.35, 34.43, 34.67, None, 62225.0, 2157182.0, None, None, None], ['2017-01-10', 34.8, 34.98, 34.46, 34.91, None, 43976.0, 1528055.0, None, None, None], ['2017-01-11', 34.95, 36.0, 34.84, 35.42, None, 123530.0, 4369079.0, None, None, None], ['2017-01-12', 35.38, 35.38, 34.31, 34.9, None, 163860.0, 5703427.0, None, None, None], ['2017-01-13', 34.98, 34.98, 34.6, 34.85, None, 59367.0, 2065534.0, None, None, None]]


In [9]:
#Get a list of the columns names to make tracking easier
col_names=AFX_X_data['dataset_data']['column_names']
print(col_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [10]:
#dump as a csv to save this dataset
test_dump = 'artefacts/AFX_X_2017.csv'
with open(test_dump, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(col_names)
    writer.writerows(data)

In [11]:
#Calculate what the highest and lowest opening prices were for the stock in this period.
#open is the second item in the list (index = 1)
# Extract opening prices and filter out None values
opening_prices = [sublist[1] for sublist in data if sublist[1] is not None]

# Find the highest and lowest opening prices
highest_opening = max(opening_prices)
lowest_opening = min(opening_prices)

print(f'Highest opening price: {highest_opening}')
print(f'Lowest opening price: {lowest_opening}')

Highest opening price: 53.11
Lowest opening price: 34.0


In [12]:
#What was the largest change in any one day (based on High and Low price)?
#High is index 2, Low is index 3
new_list = [row[2] - row[3] for row in data]
largest_change=max(new_list)
print(f'The largest change in one day is {largest_change:.2f}')

The largest change in one day is 2.81


In [13]:
#What was the largest change between any two days (based on Closing Price)?
#close = index 4
# Extract closing prices
closing_prices = [sublist[4] for sublist in data]

# Calculate changes between consecutive days
changes = [abs(j - i) for i, j in zip(closing_prices[:-1], closing_prices[1:])]

# Find the largest change
largest_change = max(changes)

print(f'The largest change between any two days is {largest_change:.2f}')

The largest change between any two days is 2.56


In [14]:
#What was the average daily trading volume during this year?
#volume is index 6
trading_vol = [sublist[6] for sublist in data]
average_trading_vol = sum(trading_vol) / len(trading_vol)
print(f"The average daily trading volume is {average_trading_vol:.2f}")


The average daily trading volume is 89124.34


In [15]:
7. #(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
trading_vol.sort()
length = len(trading_vol)
middle = length // 2

if length % 2 == 0:
    median_trading_vol = (trading_vol[middle - 1] + trading_vol[middle]) / 2
else:
    median_trading_vol = trading_vol[middle]

print(f"The median trading volume is {median_trading_vol:.2f}")

The median trading volume is 76286.00
